<a href="https://colab.research.google.com/github/chefPony/nn_zero_to_hero/blob/master/build_makemore_backprop_ninja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## makemore: becoming a backprop ninja

In [1]:
# there no change change in the first several cells from last lecture

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-07-20 17:57:10--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.006s  

2024-07-20 17:57:11 (34.0 MB/s) - ‘names.txt’ saved [228145/228145]



In [4]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# ok biolerplate done, now we get to the action:

In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3389, grad_fn=<NegBackward0>)

## Notation

$$
n = batch\ size\\
n_v = vocab\ size\\
i = 1, ..., n\\
j = 1, 2, ..., n_v\\
$$

### Derivative with respect to logprobs
$$
z_{ij} = log(p_{ij})\\
L(\mathbf{Z}, \mathbf{Y})=-\frac{1}{n}\sum_{i=1}^{n}\sum_{j=1}^{n_v}z_{ij}y_{ij}\\
\frac{\partial{L}}{\partial{z_{ij}}} = -\frac{1}{n}y_{ij}
$$


#### Computational graph
```
logprobs --  L
```



In [12]:
Ymat = torch.zeros((n, vocab_size))
Ymat[range(n), Yb] = 1.
dlogprobs = - 1/n * Ymat

### Derivative with respect to probs

$$
\frac{\partial{L}}{\partial{p_{ij}}}=\frac{\partial{L}}{\partial{z_{ij}}}\frac{\partial{z_{ij}}}{\partial{p_{ij}}}\\
\frac{\partial{log(p_{ij})}}{\partial{p_{ij}}} = \frac{1}{p_{ij}}
$$

#### Computational graph
```
probs --  logprobs
```

In [13]:
dprobs = dlogprobs * 1./probs

### Derivative with respect to counts_sum_inv
$$
\mathbf{P}(\mathbf{cs}^{inv})=
  \left[ {\begin{array}{cccc}
    p_{11}(cs_1^{inv}) & p_{12}(cs_1^{inv}) & \cdots & p_{1n_{v}}(cs_1^{inv})\\
    p_{21}(cs_2^{inv}) & p_{22}(cs_2^{inv}) & \cdots & p_{2n_{v}}(cs_2^{inv})\\
    \vdots & \vdots & \ddots & \vdots\\
    p_{n1}(cs_n^{inv}) & p_{n2}(cs_n^{inv}) & \cdots & p_{nn_{v}}(cs_n^{inv})\\
  \end{array} } \right]\\
\frac{\partial{p_{kq}}}{\partial{cs_{i}^{inv}}}=\frac{\partial{(c_{kq} \cdot cs_{k}^{inv})}}{cs_i^{inv}} =
\begin{cases}
  c_{iq} & \text{if } i=k\\    
  0      & \text{if } i \neq k\\
\end{cases}\\
\frac{\partial{L(\mathbf{P}(cs_i^{inv}))}}{\partial{cs_i^{inv}}}=
\sum_{k=1}^{n}\sum_{q=1}^{n_v}\frac{\partial{L}}{\partial{p_{kq}}}\frac{\partial{p_{kq}}}{\partial{cs_i^{inv}}}=
\sum_{q=1}^{n_v}\frac{\partial{L}}{\partial{p_{iq}}}c_{iq}\\
$$

#### Computational graph
```
counts_sum_inv --  probs
```

In [14]:
dcounts_sum_inv = (dprobs * Ymat * counts).sum(1, keepdim=True)

### Derivative with respect to counts_sum
$$
\mathbf{cs^{inv}} = [cs_1^{inv}(cs_1), \cdots, cs_n^{inv}(cs_n)]\\
\frac{\partial{cs_j^{inv}}}{\partial{cs_i}}=\frac{\partial{(cs_j^{-1})}}{\partial{cs_i}}
\begin{cases}
  -cs_{i}^{-2} & \text{if } i=j\\    
  0      & \text{if } i \neq j\\
\end{cases}\\
\frac{\partial{L(\mathbf{cs^{inv}}(cs_i))}}{\partial{cs_i}}=
\sum_{j=1}^n\frac{\partial{L}}{\partial{cs_j^{inv}}}\frac{\partial{cs_j^{inv}}}{\partial{cs_i}}=\frac{\partial{L}}{\partial{cs_i^{inv}}} \cdot \frac{-1}{cs_i^2}
\\
$$

#### Computational graph
```
counts_sum -- counts_sum_inv
```

In [15]:
dcounts_sum = dcounts_sum_inv * (-counts_sum**(-2))

### Derivative with respect to counts
$$
p_{kq}(c_{kq}, cs_{k}^{inv})=c_{kq} \cdot cs_k^{inv}(cs_{k}(c_{k1}, \dots, c_{kn_v}))\\
\frac{\partial{p_{kq}}}{\partial{c_{ij}}}=
  \frac{\partial{p_{kq}}}{\partial{c_{kq}}} \cdot
  \frac{\partial{c_{kq}}}{\partial{c_{ij}}} +
  \frac{\partial{p_{kq}}}{\partial{cs_k^{inv}}} \cdot
  \frac{\partial{cs_k^{inv}}}{\partial{cs_k}} \cdot
  \frac{\partial{cs_k}}{\partial{c_{ij}}} = \\
  \frac{\partial{p_{kq}}}{\partial{c_{ij}}} +
  \frac{\partial{p_{kq}}}{\partial{cs_k}} \cdot
  \frac{\partial{cs_k}}{\partial{c_{ij}}} = \\
\begin{cases}
  cs_i^{inv} + \frac{\partial{p_{ij}}}{\partial{cs_i}} \cdot 1
  & \text{if } i=k\ and\ j=q \\
  0 + \frac{\partial{p_{iq}}}{\partial{cs_i}} \cdot 1
  & \text{if } i=k\ and\ j \neq q\\
  0 & otherwise
\end{cases}
\\
\frac{\partial{L}}{\partial{c_{ij}}}=
\sum_{k=1}^{n}
  \sum_{q=1}^{n_v}
    \frac{\partial{L}}{\partial{p_{kq}}} \cdot
    \frac{\partial{p_{kq}}}{\partial{c_{ij}}} =
\sum_{q=1}^{n_v}\frac{\partial{L}}{\partial{p_{iq}}} \cdot (
  cs_i^{inv} +
  \frac{\partial{p_{iq}}}{\partial{cs_i}} \cdot
  \frac{\partial{cs_i}}{\partial{c_{ij}}}) = \\
\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{p_{iq}}} \cdot cs_i^{inv} +
  \frac{\partial{L}}{\partial{cs_i}} \cdot
  \frac{\partial{cs_i}}{\partial{c_{ij}}}}=\\
\frac{\partial{L}}{\partial{p_{ij}}} \cdot cs_i^{inv} +
\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{cs_i}} \cdot 1
  }
$$

Sums are gradient distributors:

$$
f(\mathbf{x}) = \sum_{i=1}^nx_i\\
[
  \frac{\partial{f(\mathbf{x})}}{\partial{x_n}},
  \cdots ,
  \frac{\partial{f(\mathbf{x})}}{\partial{x_n}}] = [1, \cdots, 1]
$$

#### Computational graph
```
counts -- probs
 |
 --- counts_sum
```

In [16]:
dcounts = dprobs * counts_sum_inv
dcounts += dcounts_sum * torch.ones_like(counts)

### Derivative with respect to norm_logits
$$
nl_{ij} = \text{normalized logits}\\
c_{ij} = e^{nl_{ij}} \\
\frac{\partial{c_{ij}}}{\partial{nl_{ij}}} = e^{nl_{ij}} = c_{ij}\\
\frac{\partial{L}}{\partial{nl_{ij}}}=
  \frac{\partial{L}}{\partial{c_{ij}}} \cdot
  \frac{\partial{c_{ij}}}{\partial{nl_{ij}}} =
  \frac{\partial{L}}{\partial{c_{ij}}} \cdot
  c_{ij}
$$

#### Computational graph
```
norm_logits -- counts
```

In [17]:
dnorm_logits = dcounts * counts

### Derivative with respect to logit_maxes
$$
nl_{ij} = l_{ij} - l_i^{max}\\
\frac{\partial{nl_{ij}}}{\partial{l_i^{max}}}=-1\\
\frac{\partial{L}}{\partial{l_i^{max}}}=
  \sum_{k=1}^n\sum_{j=1}^{n_v}\frac{\partial{L}}{\partial{nl_{kj}}}\frac{\partial{nl_{kj}}}{\partial{l_i^{max}}}=
  \sum_{j=1}^{n_v}\frac{\partial{L}}{\partial{nl_{ij}}}\frac{\partial{nl_{ij}}}{\partial{l_i^{max}}}=
  -\sum_{j=1}^{n_v}\frac{\partial{L}}{\partial{nl_{ij}}}
$$


#### Computational graph
```
logit_maxes -- logits
```



In [18]:
dlogit_maxes = -dnorm_logits.sum(1, keepdim=True)

### Derivative with respect to logits
$$
\mathbf{nl}=  \left[ {\begin{array}{cccc}
    nl_{11} & nl_{12} & \cdots & nl_{1n_{v}}\\
    nl_{21} & nl_{22} & \cdots & nl_{2n_{v}}\\\\
    \vdots & \vdots & \ddots & \vdots\\
    nl_{n1} & nl_{n2} & \cdots & nl_{nn_{v}}\\\\
  \end{array}} \right]\\
\\
\frac{\partial{nl_{kq}(l_{kq}, l_k^{max})}}{\partial{l_{ij}}} =
  \frac{\partial{l_{kq}}}{\partial{l_{ij}}} +
  \frac{\partial{nl_{kq}}}{\partial{l_k^{max}}}
  \frac{\partial{l_k^{max}}}{\partial{l_{ij}}}
\\
\frac{\partial{L(\mathbf{nl})}}{\partial{l_{ij}}}=
  \sum_{k=1}^n\sum_{q=1}^{n_v}{
    \frac{\partial{L}}{\partial{nl_{kq}}}\frac{\partial{nl_{kq}}}{\partial{l_{ij}}}=
    \sum_{q=1}^{n_v}\frac{\partial{L}}{\partial{nl_{iq}}}\frac{\partial{nl_{iq}}}{\partial{l_{ij}}}=
    \sum_{q=1}^{n_v}\frac{\partial{L}}{\partial{nl_{iq}}}(
      \frac{\partial{l_{iq}}}{\partial{l_{ij}}} +
      \frac{\partial{nl_{iq}}}{\partial{l_i^{max}}} ⋅
      \frac{\partial{l_i^{max}}}{\partial{l_{ij}}}
    )=\\
    \sum_{q=1}^{n_v}{
      \frac{\partial{L}}{\partial{nl_{iq}}} \cdot
      \frac{\partial{l_{iq}}}{\partial{l_{ij}}} +
      \frac{\partial{L}}{\partial{l_i^{max}}} ⋅
      \frac{\partial{l_i^{max}}}{\partial{l_{ij}}}
      }
  }\\
$$

#### Max is a  gradient router
$$
\frac{\partial{l_i^{max}}}{\partial{l_{ij}}}=
\begin{cases}
  1  & \text{if } l_{ij}=l_i^{max}\\
  0  & \text{otherwise}
\end{cases}
$$


#### Computational graph
```
logits -- normalized_logits
  |                            
  ------------- logit_maxes
```



In [19]:
dlogits = dnorm_logits * torch.ones_like(logits)
dlogits += dlogit_maxes * (logits == logit_maxes).float()

## Derivative with respect to h
$$
l_{kq} = \sum_{r=1}^m h_{kr} \cdot w_{rq} + b_k\\
\frac{\partial{l_{kq}}}{\partial{h_{ij}}}=
\begin{cases}
  w_{jq} & \text{if } k=i\\
  0 & \text{otherwise}
\end{cases}\\
\frac{\partial{L}}{\partial{h_{ij}}} =
\sum_{k=1}^n\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{l_{kq}}}
  \frac{\partial{l_{kq}}}{\partial{h_{ij}}}
}=
\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{l_{iq}}}
  \frac{\partial{l_{iq}}}{\partial{h_{ij}}}
}=
\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{l_{iq}}}
  w_{jq}
}
$$

#### Computational graph
```
h -- logits
```

In [20]:
dh = dlogits @ W2.T

## Derivative with respect to w2
$$
l_{kq} = \sum_{r=1}^m h_{kr} \cdot w_{rq} + b_k\\
\frac{\partial{l_{kq}}}{\partial{w_{ij}}}=
\begin{cases}
  h_{ki} & \text{if } q=j\\
  0 & \text{otherwise}
\end{cases}\\
\frac{\partial{L}}{\partial{w_{ij}}} =
\sum_{k=1}^n\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{l_{kq}}}
  \frac{\partial{l_{kq}}}{\partial{w_{ij}}}
}=
\sum_{k=1}^{n}{
  \frac{\partial{L}}{\partial{l_{kj}}}
  \frac{\partial{l_{kj}}}{\partial{w_{ij}}}
}=
\sum_{k=1}^{n}{
  \frac{\partial{L}}{\partial{l_{kj}}}
  h_{ki}
}
$$

#### Computational graph
```
W2 -- logits
```

In [21]:
dW2 = h.T @ dlogits

## Derivative with respect to b2
$$
l_{kq} = \sum_{r=1}^m h_{kr} \cdot w_{rq} + b_q\\
\frac{\partial{l_{kq}}}{\partial{b_i}}=
\begin{cases}
  1 & \text{if } q=i\\
  0 & \text{otherwise}
\end{cases}\\
\frac{\partial{L}}{\partial{b_i}} =
\sum_{k=1}^n\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{l_{kq}}}
  \frac{\partial{l_{kq}}}{\partial{b_i}}
}=
\sum_{k=1}^{n}{
  \frac{\partial{L}}{\partial{l_{ki}}}
  \frac{\partial{l_{ki}}}{\partial{b_i}}
}=
\sum_{k=1}^{n}{
  \frac{\partial{L}}{\partial{l_{ki}}}
}
$$

#### Computational graph
```
b2 -- logits
```

In [22]:
db2 = dlogits.sum(0, keepdim=True)

### Derivative with respect to hpreact

$$
\frac{\partial{h_{ij}}}{\partial{hp_{ij}}}=
  \frac{\partial{tanh(hp_{ij})}}{\partial{hp_{ij}}}=
  1 - h_{ij}^2
$$


#### Computational graph
```
hpreact -- h
```

In [23]:
dhpreact = dh * (1.0 - h**2)

###  Derivative with respect to bgain

$$
\frac{\partial{hp_{ij}}}{\partial{bg_j}}=
  \frac{\partial{(bnr_{ij} \cdot bg_j + bb_j)}}{\partial{bg_j}}=bnr_{ij}\\
\frac{\partial{L}}{\partial{bg_j}}=\sum_{k=1}^{n}\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{hp_{kq}}}\frac{\partial{hp_{kq}}}{\partial{bg_j}}=
  \sum_{k=1}^n\frac{\partial{L}}{\partial{hp_{kj}}}\frac{\partial{hp_{kj}}}{\partial{bg_j}}=
  \sum_{k=1}^n\frac{\partial{L}}{\partial{hp_{kj}}}bnr_{kj}
}
$$

#### Computational graph
```
bngain -- hpreact
```

In [24]:
dbngain = (dhpreact * bnraw).sum(0, keepdim=True)

### Derivative of bnbias

$$
\frac{\partial{hp_{ij}}}{\partial{bb_j}}=
  \frac{\partial{(bnr_{ij} \cdot bg_j + bb_j)}}{\partial{bb_j}}=1\\
\frac{\partial{L}}{\partial{bg_j}}=\sum_{k=1}^{n}\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{hp_{kq}}}\frac{\partial{hp_{kq}}}{\partial{bb_j}}=
  \sum_{k=1}^n\frac{\partial{L}}{\partial{hp_{kj}}}\frac{\partial{hp_{kj}}}{\partial{bb_j}}=
  \sum_{k=1}^n\frac{\partial{L}}{\partial{hp_{kj}}}
}
$$

#### Computational graph
```
bnbias -- hpreact
```

In [25]:
dbnbias = dhpreact.sum(0, keepdim=True)

### Derivative with respect  to bnraw


$$
\frac{\partial{hp_{ij}}}{\partial{bnr_{ij}}}=
  \frac{\partial{(bnr_{ij} \cdot bg_j + bb_j)}}{\partial{bnr_{ij}}}=bg_j\\
\frac{\partial{L}}{\partial{bnr_{ij}}}=\sum_{k=1}^{n}\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{hp_{kq}}}\frac{\partial{hp_{kq}}}{\partial{bnr_{ij}}}=
  \frac{\partial{L}}{\partial{hp_{ij}}}\frac{\partial{hp_{ij}}}{\partial{bnr_{ij}}}
}
$$

#### Computational graph
```
bnraw -- hpreact
```

In [30]:
dbnraw = dhpreact * bngain

### Derivative with respect  to bnvar_inv


$$
\frac{\partial{bnr_{ij}}}{\partial{bnv_j^{inv}}}=
  \frac{\partial{(bnd_{ij} \cdot bnv_j^{inv})}}{\partial{bnv_j^{inv}}}=bnd_{ij}\\
\frac{\partial{L}}{\partial{bnv_j^{inv}}}=\sum_{k=1}^{n}\sum_{q=1}^{n_v}{
  \frac{\partial{L}}{\partial{bnr_{kq}}}\frac{\partial{bnr_{kq}}}{\partial{bnv_j^{inv}}}=
  \sum_{k=0}^n{
    \frac{\partial{L}}{\partial{bnr_{kj}}}\frac{\partial{bnr_{kj}}}{\partial{bnv_j^{inv}}}
  }=
  \sum_{k=0}^n{
    \frac{\partial{L}}{\partial{bnr_{kj}}}bnd_{kj}
  }
}
$$

#### Computational graph
```
bnvar_inv -- bnraw
```

In [35]:
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim=True)

### Derivative with respect  to bnvar


$$
\frac{\partial{bnv_j^{inv}}}{\partial{bnv_j}}=
  \frac{\partial{(bnv_j + 1e^{-5})^{-0.5}}}{\partial{bnv_j}}=
  -0.5 \cdot (bnv_j + 1e^{-5})^{-1.5} \cdot 1\\
\frac{\partial{L}}{\partial{bnv_j}}=
\sum_{k=1}^{n_v}
{
  \frac{\partial{L}}{\partial{bnv_k^{inv}}}\frac{\partial{bnv_k^{inv}}}{\partial{bnv_j}}
}=
\frac{\partial{L}}{\partial{bnv_j^{inv}}}\frac{\partial{bnv_j^{inv}}}{\partial{bnv_j}}
$$

#### Computational graph
```
bnvar -- bnvar_inv
```

In [38]:
dbnvar = dbnvar_inv  * (-0.5) * (bnvar + 1e-5)**(-1.5)

### Derivative w.r.t bndiff2

$$
\frac{\partial{bnv_j}}{\partial{bnd^2_{ij}}}=
  \frac{\partial{(\frac{1}{n-1}\sum_{k=0}^n{bnd^2_{kj}})}}{\partial{bnd^2_{ij}}}=
  \frac{1}{n-1}\\
\frac{\partial{L}}{\partial{bnd_{ij}^2}}=
\sum_{k=1}^{n_v}
{
  \frac{\partial{L}}{\partial{bnv_k}}\frac{\partial{bnv_k}}{\partial{bnd_{ij}^2}}
}=
\frac{\partial{L}}{\partial{bnv_j}}\frac{\partial{bnv_j}}{\partial{bnd_{ij}^2}
}=
\frac{\partial{L}}{\partial{bnv_j}} \cdot \frac{1}{n-1}
$$

#### Computational graph
```
bndiff2 -- bnvar
```

In [49]:
dbndiff2 = dbnvar  * 1./(n-1) * torch.ones_like(bndiff2)

### Derivative w.r.t bndiff

$$
\frac{\partial{bnr_{ij}}}{\partial{bnd_{ij}}}=
  \frac{\partial{(bnd_{ij} \cdot bnv_j^{inv})}}{\partial{bnd_{ij}}}=bnv_j^{inv}\\
\frac{\partial{bnd^2_{ij}}}{\partial{bnd_{ij}}}=2bnd_{ij}\\
$$

#### Computational graph
```
bndiff -- bnraw
  |
  ------- bndiff2
```

In [57]:
dbndiff = dbnraw * bnvar_inv
dbndiff += dbndiff2 * 2 * bndiff

### Derivative w.r.t bnmeani

$$
\frac{\partial{bnd_{ij}}}{\partial{bnm_j}}=
  \frac{\partial{(hpb_{ij} - bnm_j)}}{\partial{bnm_j}}=-1\\
\frac{\partial{L}}{\partial{bnm_j}}=
\sum_{k=1}^{n}
{
  \frac{\partial{L}}{\partial{bnd_{kj}}}\frac{\partial{bnd_{kj}}}{\partial{bnm_j}}
}=
-\sum_{k=1}^{n}
{
  \frac{\partial{L}}{\partial{bnd_{kj}}}
}
$$

#### Computational graph
```
bnmeani -- bndiff
```

In [60]:
hprebn.shape

torch.Size([32, 64])

In [58]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
#cmp('hprebn', dhprebn, hprebn)
#cmp('embcat', dembcat, embcat)
#cmp('W1', dW1, W1)
#cmp('b1', db1, b1)
#cmp('emb', demb, emb)
#cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw  

In [ ]:
logits.max(dim=1)[1]

tensor([ 1,  2, 19, 15, 15, 25, 16,  3, 19,  8, 15,  3, 22, 18, 17,  5,  2,  7,
        22, 19, 15, 19, 22, 22, 23,  5, 22, 20, 24,  8, 24,  4])

In [ ]:
demb.shape

torch.Size([32, 3, 10])

In [ ]:
demb[Xb].shape

torch.Size([32, 3, 3, 10])

In [ ]:
C.grad.data.shape

torch.Size([27, 10])

In [ ]:
(dbndiff2 * 2 * bndiff ).shape

torch.Size([32, 64])

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = None # TODO. my solution is 3 lines
# -----------------

#cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = None # TODO. my solution is 1 (long) line
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  # YOUR CODE HERE :)
  dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    #p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  if i >= 100: # TODO: delete early breaking when you're ready to train the full net
    break

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))